In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pydicom 

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import PIL
from PIL import Image
import matplotlib
from keras.preprocessing.image import img_to_array
from numpy import expand_dims
from keras.preprocessing.image import load_img

In [ ]:
image1 = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')

In [ ]:
train = pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')

In [ ]:
image1.describe()

In [ ]:
column = image1['image_id']
print(column)
print (image1)
index = np.arange(0,len(column))
print(index)

In [ ]:
image1 = image1.drop_duplicates(keep='last',subset = ["image_id"])
print((image1.shape))
im1 = image1['image_id']
im1 = np.array(im1)
print(im1[0])

In [ ]:
im1 = image1['image_id']

In [ ]:
#index = np.array(column[1:len(column),0])
from keras.preprocessing.image import load_img
import cv2 
index = np.arange(0,10000)
x=np.array([])
data=[]
for im in index:
    dicom = pydicom.read_file('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train' + "/"+ im1[im] + ".dicom" )
    image = np.array(dicom.pixel_array)
    image = cv2.resize(image,(416,416))
    data.append(image)
    #print (type(image))

data = np.array(data)

In [ ]:
print(im)
print(index)

In [ ]:
index1 = np.arange(10000,15000)
x=np.array([])
data1=[]
for i in index1:
    dicom1 = pydicom.read_file('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train' + "/"+ im1[i] + ".dicom" )
    image1 = np.array(dicom1.pixel_array)
    image1 = cv2.resize(image1,(416,416))
    data1.append(image1)
    #print (type(image))

data1 = np.array(data1)

In [ ]:
print ()

In [ ]:
input_data1 = np.resize(data1,(500,416,416,3))/255
#print(input_data1)
print (input_data1)

In [ ]:
print(data.shape)
input_data = np.resize(data,(1000,416,416,3))/255
print(input_data.shape)
print(input_data1.shape)

In [ ]:
from numpy import expand_dims
print(data.shape)
data = expand_dims(data, -1)
print(data.shape)
#data = np.array(data)

In [ ]:
#print(dicom)
#print (data.shape)
plt.imshow(data[2], cmap='gray')
#print(x[0])

print (image1)
print(data[1].shape)
print(data.shape)
print(type(data))

In [ ]:
#image1.reset_index()
column1 = image1['class_id']
column1 = np.array(column1)
print (type(column1))
labels = [None] * len(column1)
la = []
#print (labels)
#labels = np.array(range(len(column1)), dtype=str)
index = np.arange(0,len(column1),1)
#print ((index))
for i in index:
    if column1[i] == 14:
        labels[i] = 1
       
    else:
        labels[i] = 0
        #print (labels[i])
        #print (i)
        
print(len(labels))        

In [ ]:
df = pd.DataFrame(labels,columns = ['new'])

df1 = pd.DataFrame(column1)
df1.columns= ['old']

print (df.shape)
print (df1.shape)
#df.reset_index(drop=True, inplace=True)
#df1.reset_index(drop=True, inplace=True)
#print(df)
#print(df1)
y = column1.to_frame()
final  = pd.concat([df,df1],axis=1)
#print(final)
comparison_column = np.where(final["new"] == final["old"], True, False)
final["equal"] = comparison_column

print(final)

In [ ]:
#column1 = image1['class_id']
#labels = [None] * len(column1)
#index = np.arange(1,len(column1),1)
#print ((index))
#for i in index:
 #   if column1[i] == 14:
  #      labels[i] = column1[i]
  #  else:
   #     labels[i] == "Disease"

In [ ]:
print (labels[1])

In [ ]:
y = image1['class_id']
y1 = y.iloc[:10]
print(y1)


In [ ]:

# create a YOLOv3 Keras model and save it to file
# based on https://github.com/experiencor/keras-yolo3
import struct
import numpy as np
from keras.layers import Conv2D
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import ZeroPadding2D
from keras.layers import UpSampling2D
from keras.layers.merge import add, concatenate
from keras.models import Model

def _conv_block(inp, convs, skip=True):
	x = inp
	count = 0
	for conv in convs:
		if count == (len(convs) - 2) and skip:
			skip_connection = x
		count += 1
		if conv['stride'] > 1: x = ZeroPadding2D(((1,0),(1,0)))(x) # peculiar padding as darknet prefer left and top
		x = Conv2D(conv['filter'],
				   conv['kernel'],
				   strides=conv['stride'],
				   padding='valid' if conv['stride'] > 1 else 'same', # peculiar padding as darknet prefer left and top
				   name='conv_' + str(conv['layer_idx']),
				   use_bias=False if conv['bnorm'] else True)(x)
		if conv['bnorm']: x = BatchNormalization(epsilon=0.001, name='bnorm_' + str(conv['layer_idx']))(x)
		if conv['leaky']: x = LeakyReLU(alpha=0.1, name='leaky_' + str(conv['layer_idx']))(x)
	return add([skip_connection, x]) if skip else x

def make_yolov3_model():
	input_image = Input(shape=(None, None, 3))
	# Layer  0 => 4
	x = _conv_block(input_image, [{'filter': 32, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 0},
								  {'filter': 64, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 1},
								  {'filter': 32, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 2},
								  {'filter': 64, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 3}])
	# Layer  5 => 8
	x = _conv_block(x, [{'filter': 128, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 5},
						{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 6},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 7}])
	# Layer  9 => 11
	x = _conv_block(x, [{'filter':  64, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 9},
						{'filter': 128, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 10}])
	# Layer 12 => 15
	x = _conv_block(x, [{'filter': 256, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 12},
						{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 13},
						{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 14}])
	# Layer 16 => 36
	for i in range(7):
		x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 16+i*3},
							{'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 17+i*3}])
	skip_36 = x
	# Layer 37 => 40
	x = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 37},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 38},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 39}])
	# Layer 41 => 61
	for i in range(7):
		x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 41+i*3},
							{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 42+i*3}])
	skip_61 = x
	# Layer 62 => 65
	x = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 2, 'bnorm': True, 'leaky': True, 'layer_idx': 62},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 63},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 64}])
	# Layer 66 => 74
	for i in range(3):
		x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 66+i*3},
							{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 67+i*3}])
	# Layer 75 => 79
	x = _conv_block(x, [{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 75},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 76},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 77},
						{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 78},
						{'filter':  512, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 79}], skip=False)
	# Layer 80 => 82
	yolo_82 = _conv_block(x, [{'filter': 1024, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 80},
							  {'filter':  255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 81}], skip=False)
	# Layer 83 => 86
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 84}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_61])
	# Layer 87 => 91
	x = _conv_block(x, [{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 87},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 88},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 89},
						{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 90},
						{'filter': 256, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True, 'layer_idx': 91}], skip=False)
	# Layer 92 => 94
	yolo_94 = _conv_block(x, [{'filter': 512, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 92},
							  {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 93}], skip=False)
	# Layer 95 => 98
	x = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True, 'leaky': True,   'layer_idx': 96}], skip=False)
	x = UpSampling2D(2)(x)
	x = concatenate([x, skip_36])
	# Layer 99 => 106
	yolo_106 = _conv_block(x, [{'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 99},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 100},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 101},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 102},
							   {'filter': 128, 'kernel': 1, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 103},
							   {'filter': 256, 'kernel': 3, 'stride': 1, 'bnorm': True,  'leaky': True,  'layer_idx': 104},
							   {'filter': 255, 'kernel': 1, 'stride': 1, 'bnorm': False, 'leaky': False, 'layer_idx': 105}], skip=False)
	model = Model(input_image, [yolo_82, yolo_94, yolo_106])
	return model

class WeightReader:
	def __init__(self, weight_file):
		with open(weight_file, 'rb') as w_f:
			major,	= struct.unpack('i', w_f.read(4))
			minor,	= struct.unpack('i', w_f.read(4))
			revision, = struct.unpack('i', w_f.read(4))
			if (major*10 + minor) >= 2 and major < 1000 and minor < 1000:
				w_f.read(8)
			else:
				w_f.read(4)
			transpose = (major > 1000) or (minor > 1000)
			binary = w_f.read()
		self.offset = 0
		self.all_weights = np.frombuffer(binary, dtype='float32')

	def read_bytes(self, size):
		self.offset = self.offset + size
		return self.all_weights[self.offset-size:self.offset]

	def load_weights(self, model):
		for i in range(106):
			try:
				conv_layer = model.get_layer('conv_' + str(i))
				print("loading weights of convolution #" + str(i))
				if i not in [81, 93, 105]:
					norm_layer = model.get_layer('bnorm_' + str(i))
					size = np.prod(norm_layer.get_weights()[0].shape)
					beta  = self.read_bytes(size) # bias
					gamma = self.read_bytes(size) # scale
					mean  = self.read_bytes(size) # mean
					var   = self.read_bytes(size) # variance
					weights = norm_layer.set_weights([gamma, beta, mean, var])
				if len(conv_layer.get_weights()) > 1:
					bias   = self.read_bytes(np.prod(conv_layer.get_weights()[1].shape))
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel, bias])
				else:
					kernel = self.read_bytes(np.prod(conv_layer.get_weights()[0].shape))
					kernel = kernel.reshape(list(reversed(conv_layer.get_weights()[0].shape)))
					kernel = kernel.transpose([2,3,1,0])
					conv_layer.set_weights([kernel])
			except ValueError:
				print("no convolution #" + str(i))

	def reset(self):
		self.offset = 0

# define the model
model = make_yolov3_model()
# load the model weights
weight_reader = WeightReader('/kaggle/input/rrmqptest2/second/yolov3.weights')
# set the model weights into the model
weight_reader.load_weights(model)
# save the model to file
model.save('model.h5')
print (model.summary())

In [ ]:
model.summary()

In [ ]:
model.outputs[0]
#model.layers[-1]

In [ ]:
from tensorflow.keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D
#print (model.layers[-1].output)
x = GlobalMaxPooling2D(name = 'global')(model.layers[-1].output)
output= Dense(1,activation = 'sigmoid')(x)
inputs= model.inputs
model_1 = Model(inputs= inputs, outputs= output)
#print (inputs)

In [ ]:
model_1.summary()

In [ ]:
for layer in model_1.layers:
    layer.trainable = False
model_1.summary()

In [ ]:
model_1.layers[-2].trainable= True
model_1.layers[-3].trainable= True
model_1.layers[-5].trainable= True
model_1.summary()

In [ ]:
#print(labels)
la = labels[:1000]
la1 = labels[1000:1500]

la = np.array(la)
la1=np.array(la1)
print(la.shape)
#print(dtype(la))
y_train = np.asarray(la).astype('float32').reshape((-1,1))
print(y_train)

In [ ]:
model_1.compile(optimizer = 'adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model_1.fit(input_data,la,epochs=15,validation_data=(input_data1,la1))


In [ ]:
print(history.history)
plt.figure()
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training and Testing Accuracy per epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training','Testing'])
plt.show()

In [ ]:
print(history.history)
plt.figure()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training and Testing Loss per epoch')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Training','Testing'])
plt.show()

In [ ]:
yo = model_1.predict(input_data1)

In [ ]:
print(input_data.shape)
print (yo)

In [ ]:
matplotlib.image.imsave('xray1.png', input_data[0])

In [ ]:
# load yolov3 model
from keras.models import load_model

model = load_model('model.h5')

In [ ]:
#im = Image.fromarray(data[2])
#im.save("xray.png")

In [ ]:
matplotlib.image.imsave('xray1.png', data[2])

In [ ]:
#yhat = model.predict(image)

In [ ]:
#print([a.shape for a in yhat])

In [ ]:
print (image1['x_min']) 